Znajdowanie najkrótszych ścieżek w grafach ważonych

Reprezentacja grafów ważonych:
    </br></br>
    $G=(V,E)$ </br>
    $w=E \to N (\Z ,\R \to \mathbb{Q}) $ </br>
1. Reprezentacja macierzowa: </br>
$W$ - macierz wag </br>
$W[i][j]$ - waga krawędzi między wierzchołkami $v_i$ oraz $v_j$ ($\infty$ jeżeli nie istnieje) </br>
2. Reprezentacja listowa: </br>
Tak samo jak zwykła reprezentacja, przy czym na $i$-tej pozycji w liście mamy listę dwuelementową z informacją o skierowaniu na inny wierzchołek, a druga wartość to waga tego połączenia. </br>

Problem znajowania najkrótszych ścieżek:
- 1:1, problem ścieżki między dwoma wierzchołkami - na elementarnym poziomie trudne do wykorzystania
- 1:wszyscy, problem ścieżki z zadanego wierzchołka do wszystkich innych
- wszyscy:wszyscy 

Podejście elementarne/BFS:
- długości/wagi krawędzi to małe liczby naturalne
- tworzymy sztuczne wierzchołki odpowiednio dzielące krawędzie (dla wagi $n$, wkładamy $n-1$ wierzchołków, aby wydłuzyć odległości)

Implementacja: 
- wrzucamy do kolejki częściowe wierzchołki i aktualizujemy odległości, jeśli wyciągniemy prawdziwy wierzchołek.

Algorytm Dijkstry - algorytm elementarny, ale w każdym kroku skacze do najbliższego prawdziwego wierzchołka:
- liczby nie muszą być naturalne, ale nie moga być ujemne.

1. Notacja:
    - $G=(V,E)$
    - $w(u,v)$ - odległość z $u$ do $v$
    - $u.d$ - oszacowanie odległości ze źródla do $u$
    - $u.parent$ - poprzednik na najkrótszej ścieżce ze źródła

2. Algorytm: (start $=s\in V$)
    - umieść wszystkie wierzchołki w kolejce priorytetowej z oszacowaniem odległości $u.d=\infty$ (w praktyce realizowane inaczej)
    - zmień odległość $s$ na $0$
    - póki wierzchołki są w kolejce:
        - wyjmij z kolejki wierzchołek $u$ od minimalnej wartości $u.d$
        - dla każdej krawędzi $(u,v)$ wykonaj relaksację (sprawdź warunek trójkąta, czyli czy da się lepiej podejść do wierzchołka) 

3. Dowód poprawności:
    - realizuje BFS z dodanymi wierzchołkami (nieprawdiłowy argument, jeśli wagi nie są naturalne)
    - dowód przez indukcję...

4. Złożonosć:
    - $O(ElogV)$ dla reprezentacji listowej - użycie kolejki priorytetowej
    - $O(V^2)$ dla reprezentacji macierzowej - po prostu szukamy dla każdego wierzchołka

In [1]:
def Dijkstra(G,s):
    from math import inf
    from queue import PriorityQueue
    n=len(G)
    Q=PriorityQueue()
    parent=[None for _ in range(n)]
    d=[inf for _ in range(n)]
    d[s]=0
    Q.put((d[s],s))
    
    while not Q.empty():
        w,u=Q.get()
        # print(w,u)
        if w==d[u]: # jeżeli jestem na najkrótszej ścieżce
            d[u]=w
            for v,c in G[u]:
                if relax(parent,d,v,u,c):
                    Q.put((d[v],v))
        # inaczej skipuję starą ścieżkę
    return parent, d

def relax(parent,d,v,u,c): #strzelanie do ludzi 
    if d[v]>d[u]+c: # jeżeli da się lepiej podejść do wierzchołak, to to robimy
        d[v]=d[u]+c
        parent[u]=v
        return True
    return False

G=[[(1,3),(3,7),(4,8)],[(3,4),(2,1)],[],[(2,2)],[(3,3)]]

print(Dijkstra(G,0))

([4, 2, None, None, None], [0, 3, 4, 7, 8])


Algorytm Bellmana-Forda - najkrótsze ścieżki przy dopuszczeniu ujemnych wag (graf skierowany)

1. Inicjalizacja:

2. Relaksacje:

3. Wykrywanie cykli:

4. Złożoność:
    - $O(VE)$

In [ ]:
def Belford():
    pass

Najkrótsze ścieżki między każdą parą wierzchołków: </br>
    - $|V|$ wywołań Dijkstry - $O(VElogV)$ </br>
    - $|V|$ wywołań Bellmana-Forda - $O(V^2E)$ 

Algorytm Floyda-Warshalla: </br>
    - $O(V^3)$